# Encontrando insights de la UEFA Champions League

La Liga de Campeones de la UEFA, a menudo llamada Liga de Campeones, es una competencia anual de fútbol por excelencia que cautiva a los fanáticos de todo el mundo. Establecida en 1955 como la Copa de Clubes Campeones de Europa, evolucionó a la Liga de Campeones de la UEFA en 1992, ampliando su atractivo. El formato moderno cuenta con 32 equipos de clubes de primer nivel seleccionados en función de su desempeño en la liga nacional, lo que aumenta la intriga.  

Este evento electrizante trasciende los deportes y se convierte en una celebración de la unidad, la cultura y el orgullo nacional. Los fanáticos, vestidos con los colores de sus países, crean una atmósfera eléctrica, lo que hace que el torneo sea tanto sobre los espectadores como sobre los jugadores. Financieramente, la Liga de Campeones es un salvavidas para los clubes, ya que aumenta los ingresos y ofrece oportunidades transformadoras. Sin embargo, genera debates sobre las disparidades de riqueza en el fútbol europeo.  

La Liga de Campeones es sinónimo de rivalidades históricas, triunfos de los desvalidos y brillantez individual. Para los jugadores, representa la cima de su carrera, mientras que para los fanáticos, es un fenómeno cultural. El himno y los rituales icónicos enriquecen la experiencia futbolística. En 200 palabras, la UEFA Champions League es el epítome de la excelencia del fútbol europeo, que ofrece momentos inolvidables, recompensas económicas y un impacto cultural único, con 32 clubes de primer nivel que se suman a su atractivo.

**Nombres de las tablas o ficheros a utilizar**:  
[uefa_2020.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/EQuW8d16tZJKpYZN6jR480sBgc1IYypAw9hGHKQARS560g?e=7TybBU)  
[uefa_2021.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/ESG8YBrjD3tBky3aGT0MwIABNt_PQxFVVYVlWMnZYUmV8g?e=VrYHtX)  
[uefa_2022.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/Ecf8O8U8sxJLoW9Bkd2ZVzgBb_gZc35mNcprnR9FMlzliQ?e=Vn3a6v)  
- Todas las tablas tienen las mismas columnas y data types

| Column | Definition | Data type |
|--------|------------|-----------|
| `STAGE`| Stage of the March | `VARCHAR(50)` |
| `DATE` | When the match occurred. | `DATE` |
| `PENS` | Did the match end with penalty | `VARCHAR(50)` |
| `PENS_HOME_SCORE` | In case of penalty, score by home team | `VARCHAR(50)` |
| `PENS_AWAY_SCORE` | In case of penalty, score by away team | `VARCHAR(50)` |
| `TEAM_NAME_HOME` | Team home name | `VARCHAR(50)` |
| `TEAM_NAME_AWAY`| Team away  name | `VARCHAR(50)` |
| `TEAM_HOME_SCORE` | Team home score | `NUMBER` |
| `TEAM_AWAY_SCORE` | Team away score | `NUMBER` |
| `POSSESSION_HOME` | Ball possession for the home team | `FLOAT` |
| `POSSESSION_AWAY` | Ball possession for the away team | `FLOAT` |
| `TOTAL_SHOTS_HOME` | Number of shots by the home team | `NUMBER` |
| `TOTAL_SHOTS_AWAY` | Number of shots by the away team | `NUMBER`
| `SHOTS_ON_TARGET_HOME` | Total shot for home team | `FLOAT` |
| `SHOTS_ON_TARGET_AWAY` | Total shot for away team | `FLOAT` |
| `DUELS_WON_HOME` | duel win possession of ball - for home team | `NUMBER` |
| `DUELS_WON_AWAY` | duel win possession of ball - for away team | `NUMBER` 
| `PREDICTION_TEAM_HOME_WIN` | Probability of home team to win | `FLOAT` |
| `PREDICTION_DRAW` | Probability of draw | `FLOAT` |
| `PREDICTION_TEAM_AWAY_WIN` | Probability of away team to win | `FLOAT` |
| `LOCATION` | Stadium where the match was held | `VARCHAR(50)` |

### Requerimientos:  
-Puedes utilizar SQL o PySpark

In [0]:
from pyspark.sql import SparkSession

# Iniciar una sesión de Spark
spark = SparkSession.builder.appName("UEFA_Analysis").getOrCreate()

1. Encuentra los 3 equipos que anotaron más goles jugando en su estadio en la UEFA Champions League 2020-21. El resultado debe contener dos columnas: TEAM_NAME_HOME y TEAM_HOME_SCORE ordenadas en orden descendente de TEAM_HOME_SCORE. Guarda la consulta (o el notebook) como TEAM_HOME_WITH_MOST_GOALS.

Primero, cargar el archivo CSV de la temporada 2020-2021 en un DataFrame (DF):

In [0]:
# Cargar el archivo en un DF
df_2020 = spark.read.option("header", "true") \
                    .option("inferSchema", "true") \
                    .csv("dbfs:/FileStore/PySpark-Data-Cleansing/Caso_2/uefa_2020.csv")

# Mostrar el esquema y el contenido del DF
df_2020.printSchema()
df_2020.display()

root
 |-- STAGE: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- PENS: boolean (nullable = true)
 |-- PENS_HOME_SCORE: boolean (nullable = true)
 |-- PENS_AWAY_SCORE: boolean (nullable = true)
 |-- TEAM_NAME_HOME: string (nullable = true)
 |-- TEAM_NAME_AWAY: string (nullable = true)
 |-- TEAM_HOME_SCORE: integer (nullable = true)
 |-- TEAM_AWAY_SCORE: integer (nullable = true)
 |-- POSSESSION_HOME: string (nullable = true)
 |-- POSSESSION_AWAY: string (nullable = true)
 |-- TOTAL_SHOTS_HOME: integer (nullable = true)
 |-- TOTAL_SHOTS_AWAY: integer (nullable = true)
 |-- SHOTS_ON_TARGET_HOME: integer (nullable = true)
 |-- SHOTS_ON_TARGET_AWAY: integer (nullable = true)
 |-- DUELS_WON_HOME: string (nullable = true)
 |-- DUELS_WON_AWAY: string (nullable = true)
 |-- PREDICTION_TEAM_HOME_WIN: string (nullable = true)
 |-- PREDICTION_DRAW: string (nullable = true)
 |-- PREDICTION_TEAM_AWAY_WIN: string (nullable = true)
 |-- LOCATION: string (nullable = true)



STAGE,DATE,PENS,PENS_HOME_SCORE,PENS_AWAY_SCORE,TEAM_NAME_HOME,TEAM_NAME_AWAY,TEAM_HOME_SCORE,TEAM_AWAY_SCORE,POSSESSION_HOME,POSSESSION_AWAY,TOTAL_SHOTS_HOME,TOTAL_SHOTS_AWAY,SHOTS_ON_TARGET_HOME,SHOTS_ON_TARGET_AWAY,DUELS_WON_HOME,DUELS_WON_AWAY,PREDICTION_TEAM_HOME_WIN,PREDICTION_DRAW,PREDICTION_TEAM_AWAY_WIN,LOCATION
Final,29-05-2021,false,false,false,Manchester City,Chelsea,0,1,61%,39%,7,8,1,2,50%,50%,55%,3%,42%,Estádio Do Dragão
Semi-finals second leg,05-05-2021,false,false,false,Chelsea,Real Madrid,2,0,32%,68%,15,7,5,5,52%,49%,43%,3%,54%,Stamford Bridge
Semi-finals second leg,04-05-2021,false,false,false,Manchester City,PSG,2,0,44%,56%,12,14,5,0,53%,47%,44%,3%,53%,Etihad Stadium
Semi-finals first leg,28-04-2021,false,false,false,PSG,Manchester City,1,2,40%,60%,10,11,4,6,47%,53%,65%,4%,31%,Parc des Princes
Semi-finals first leg,27-04-2021,false,false,false,Real Madrid,Chelsea,1,1,51%,49%,9,11,1,5,49%,52%,61%,4%,35%,Alfredo Di Stéfano
Quarter-finals second leg,14-04-2021,false,false,false,Liverpool,Real Madrid,0,0,57%,43%,15,6,4,2,50%,50%,34%,3%,63%,Anfield
Quarter-finals second leg,14-04-2021,false,false,false,Borussia Dortmund,Manchester City,1,2,38%,62%,10,17,3,5,42%,58%,53%,3%,44%,SIGNAL IDUNA PARK
Quarter-finals second leg,13-04-2021,false,false,false,PSG,Bayern Munich,0,1,44%,56%,10,14,3,5,54%,46%,55%,3%,42%,Parc des Princes
Quarter-finals second leg,13-04-2021,false,false,false,Chelsea,FC Porto,0,1,46%,54%,8,8,1,2,52%,48%,78%,2%,20%,Ramón Sánchez-Pizjuán
Quarter-finals first leg,07-04-2021,false,false,false,FC Porto,Chelsea,0,2,39%,61%,12,6,5,3,62%,38%,26%,3%,71%,Ramón Sánchez-Pizjuán


Utilizar las funciones `groupBy`, `agg` y `sum` para agrupar las filas según el nombre del equipo local, y añadir otra columna que sea la suma de `TEAM_HOME_SCORE` para ese nombre de equipo, ess decir, el total de goles que ha anotado el equipo jugando como local.

Después, utilizar `orderBy` con las opciones `desc` y `limit(3)` para ordenar los resultados según los 3 valores más altos de `TEAM_HOME_SCORE`.

In [0]:
from pyspark.sql.functions import sum, col

# Agrupar por TEAM_NAME_HOME y sumar los goles de cada equipo en casa
df_2020 = df_2020.groupBy("TEAM_NAME_HOME").agg(sum("TEAM_HOME_SCORE").alias("TEAM_HOME_SCORE"))

# Ordenar por el total de goles en casa en orden descendente y seleccionar los primeros 3
df_2020 = df_2020.orderBy(col("TEAM_HOME_SCORE").desc()).limit(3)

df_2020.display()

TEAM_NAME_HOME,TEAM_HOME_SCORE
Manchester City,15
Real Madrid,14
Bayern Munich,13


2. Encuentra el equipo con posesión mayoritaria la mayor cantidad de veces durante la UEFA Champions League 2021-22. El resultado debe incluir dos columnas: TEAM_NAME y GAME_COUNT, que es la cantidad de veces que el equipo tuvo posesión mayoritaria durante un partido de fútbol. Guarda esta consulta (o el notebook) como TEAM_WITH_MAJORITY_POSSESSION

3. Encuentra la lista de equipos de cada fase del juego que ganaron el duelo en un partido pero terminaron perdiendo el juego en el Campeonato de la UEFA 2022-23. El resultado debe contener dos columnas: STAGE y TEAM_LOST. Guarda la consulta (o el notebook) como TEAM_WON_DUEL_LOST_GAME_STAGE_WISE.